In [ ]:
import os
import re
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.image import imread
from scipy.fft import fft, ifft
import numpy as np
from numpy import diff
import cmath
import cv2
from skimage.io import imread, imshow
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage import color, exposure, transform
from skimage.exposure import equalize_hist
from skimage.data import gravel
from skimage.filters import difference_of_gaussians, window
from scipy.fft import fftn, fftshift
from skimage.data import gravel
from skimage.filters import difference_of_gaussians, window
from scipy.fft import fftn, fftshift
import time
from scipy.signal import argrelextrema
from natsort import natsorted, ns
from tifffile import imwrite
from PIL import Image
from skimage.morphology import reconstruction
from skimage.exposure import rescale_intensity
import numpy as np
import imageio as iio
import matplotlib.image as mpimg
from matplotlib import cm
from skimage.io import imread, imshow
import cv2
from skimage.transform import hough_line, hough_line_peaks


#List all images in the directory
""" p = r'C:\\Users\\ArzinaIsmaili\\OneDrive - Adaptix Limited\\Desktop\\deconvolution_adaptix2024\\HeightRecogCode\\DowtyPrismBlade2ndHalf'

images = Path(p)
dir_list = os.listdir(images)

ext = '.tif'
res = []

for item in dir_list:
    if bool(re.findall(ext,item)):
        res.append(item)
   
res = natsorted(res, key=lambda y: y.lower())      
print(res)

 """


In [ ]:
quality = np.empty(1)
quality_data = dict()
quality_img = np.empty(1)
quality_img_data = dict()
fig, ax = plt.subplots(nrows=1, ncols=2)


for thr in range(2,3,1):
    i = 0
    print(thr)
    quality = np.empty(1)
    quality_img = np.empty(1)
    
   
    for item in res:

        start = time.time()
        path = p+'/'+item
        img = imread(path)

        # applying a bandpass filter
        filtered_image = difference_of_gaussians(img, thr)

        # window image to improve FFT (because windowing will reduce discontinuities/spectral leakage)
        wimage = img * window('hann', img.shape)  
        filtered_wimage = filtered_image * window('hann', img.shape)

        # using fftshift so that we are shifting the zero-frequency component to the center of the array
        im_f_mag = fftshift(np.abs(fftn(wimage))) # compute FFT of the original image, not needed during normal usage
        fim_f_mag = fftshift(np.abs(fftn(filtered_wimage))) # compute FFT of the filtered image, also not needed during normal usage
        
        
        save_path = p + r'/Filtered/Fil_'+ item 

        # Create then save an image from the array `filtered_wimage' (stored in the form of a numbered array)
        i = Image.fromarray(filtered_wimage)
        i.save(save_path)
        



        #hole/peak extraction from windowed image
        
        # seed image (a.k.a. marker image) specifies the values that are dilated or eroded
        seed = np.copy(wimage)
        # setting all values (except the outside border) of the seed image to be the max pixel value (255 = white)
        seed[1:-1, 1:-1] = wimage.max() 
        # mask give the maximum (dilation) / minimum (erosion) allowed value at each pixel
        mask = wimage

        # neighbouring high intensity values are replaced by low-intesity values; each seed value must be more than or equal to the 
        # corresponding mask value 
        filled = reconstruction(seed, mask, method='erosion')
   
        seed = np.copy(wimage)
        seed[1:-1, 1:-1] = wimage.min()  # smallest pixel value
        # high-intensity values will replace nearby low-intensity values; each seed value must be less than or equal to the 
        # corresponding mask value; for erosion, the reverse is true
        rec = reconstruction(seed, mask, method='dilation')
        
        peaks = wimage-rec  # this is like subtracting all teh low-intensity pixels, leaving behind the high-intensity ones e.g., edges
        
        save_path = p + r'/Filtered/peaks_'+ item 
        i = Image.fromarray(peaks)
        i.save(save_path)
        
        holes = wimage-filled  # this is like subtracting all the high-intesity pixels, leaving behind the low-intensity ones
        
        save_path = p + r'/Filtered/holes_'+ item 
        i = Image.fromarray(holes)
        i.save(save_path)
        
        #end of hole/peak extraction
        

        non_z_fft = fim_f_mag>0  # boolean 2d array – positive values of the FFT of the filtered image are set to True
        tot = sum(sum(non_z_fft))  # finding the number of True values in the 2D boolean array
        quality = np.append(quality, sum( fim_f_mag[non_z_fft] )/tot)  # the second bit: average all positive values of filtered windowed image

        non_z = img>0  # boolean 2d array - postive values of the original image are set to True
        tot_img = sum(sum(non_z))  # finding the number of True values in the 2D boolean array
        quality_img = np.append(quality_img, sum( wimage[non_z] )/tot_img)  # the second bit: average all positive values of original windowed image
        

        end = time.time()


    quality = quality[1:]  # ignore the first value because its effectively 0
    quality_data[thr] = quality  # quality_data is a dictionary
    
    quality_img = quality_img[1:]
    quality_img_data[thr] = quality_img
    
   
    
    ax[0].plot(quality,'-o',label = '%d' %thr)
    ax[0].legend()
    ax[0].set_title('Quality of filtered windowed image')

    ax[1].plot(quality_img,'-o',label = '%d' %thr)
    ax[1].legend()
    ax[1].set_title('Quality of windowed image')
    

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 20))

ax[0, 0].imshow(img, cmap='gray')
ax[0, 0].set_title('Original Image')

ax[0, 1].imshow(np.log(im_f_mag), cmap='magma')
ax[0, 1].set_title('Original FFT Magnitude (log)')

ax[1, 0].imshow(filtered_image, cmap='gray')
ax[1, 0].set_title('Filtered Image')  # bandpass filter using difference of gaussians

ax[1, 1].imshow(np.log(fim_f_mag), cmap='magma')
ax[1, 1].set_title('Filtered FFT Magnitude (log)')

plt.show()

In [ ]:
from scipy.signal import argrelextrema  # use to calculate the relative extrema of data

data = quality_data[2]
p = argrelextrema(data, comparator = np.greater, axis = 0, order = 1)  # returns a tuple of nd arrays, containing indices of the maxima in data
# np.greater takes two input arrays (of the same shape), returns a boolean array or scalar  
plt.plot(data,'-')
p = np.transpose(p)
plt.title('Image Quality Factor')
plt.plot(p, data[p],'o')  # plotting index of maxima on x-axis, then maxima value along y-axis
plt.xlabel('Index of maxima')
plt.ylabel('Value of maxima')
plt.grid()
print(p)